#Global Imports

In [1]:
%pylab inline

//anaconda/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


Populating the interactive namespace from numpy and matplotlib


###External packages

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as sp
import numpy as np
import os
import math
import pickle
from datetime import datetime
from dateutil import parser
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.model_selection import train_test_split
from sklearn import tree
from sklearn.linear_model import ElasticNet
from sklearn import linear_model
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error

In [1]:
#from keras.models import Sequential
#from keras.layers import Dense
#from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

Using Theano backend.


###Internal classes

In [5]:
#from DataProcessing import *

###Display variables

In [6]:
mpl.rcParams['pdf.fonttype'] = 42
mpl.rcParams['legend.numpoints'] = 1
sns.set_style("whitegrid")

####Global Variables

In [5]:
arc_order = ['36', '37', '38', '39', '40', '41', '42', '43', '44', '45', '46', '47', '48',
             '49', '50', '51', '52', '53', '54', '55']
depth_order = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18']#,
               #'19', '20', '21', '22']
left_right_order = ['-10', '-9', '-8', '-7', '-6', '-5', '-4', '-3', '-2', '-1', '0', '1', '2', '3', '4',
                    '5', '6', '7', '8', '9', '10']

In [4]:
player_factors = ['Angle_Avg', 'Angle_Cons', 'Depth_Avg', 'Depth_Cons', 'Left_Right_Avg', 'Left_Right_Cons']

####Functions

In [ ]:
def get_date(x):
    time_parsed = parser.parse(x)
    return '{0}_{1}_{2}'.format(time_parsed.year, time_parsed.month, time_parsed.day)

In [1]:
def round_float(x):
    return str(int(round(x)))

def square_root(x):
    return math.sqrt(x)

In [1]:
# round to half number
def round_half(x):
    return str(round(x * 2) / 2)
def round_two(x):
    return str(round(x * 5) / 5)

In [1]:
def numeric_make_miss(x):
    if x == True:
        return 1
    else:
        return 0

In [2]:
def add_columns(df):
    if 'shot_length' in df.columns:
        df['shot_distance'] = df.shot_length
        df['angle'] = df.entry_angle
        df['depth'] = df.shot_depth
    elif 'floor_position_x' in df.columns:
        df['floor_position_x_inch'] = df['floor_position_x'] / 25.4
        df['floor_position_y_inch'] = df['floor_position_y'] / 25.4
        df['floor_position_x_feet'] = df['floor_position_x_inch'] / 12
        df['floor_position_y_feet'] = df['floor_position_y_inch'] / 12
        df['shot_distance_squared'] = df['floor_position_x_feet']**2 + df['floor_position_y_feet']**2
        df['shot_distance'] = df['shot_distance_squared'].apply(square_root)
    df['angle_rounded'] = df['angle'].apply(round_float)
    df['depth_rounded'] = df['depth'].apply(round_float)
    df['left_right_rounded'] = df['left_right'].apply(round_float)
    df['numeric_make_miss'] = df['make_miss'].apply(numeric_make_miss)
    #df['make_miss'] = df['make_miss'].apply(fix_make_miss)
    #df['accurate'] = df['shot_date'].apply(get_accurate)
    #df['clean_make'] = df['clean_make'].apply(fix_make_miss)
    #df['make_miss'] = df[['make_miss', 'clean_make']].apply(make_miss_int, axis=1)
    return df

In [21]:
def get_accurate(x):
    if int(x.split('-')[1]) >= 5:
        return True
    else:
        return False

In [19]:
def fix_make_miss(x):
    if x == True or x == 'true':
        return True
    elif x == False or x == 'false':
        return False

In [16]:
def make_miss_int(x):
    if x[0] == True or x[1] == True:
        return 1
    else:
        return 0

####large scale

In [58]:
def angle(df, df_ft):
    # Round angle
    df['angle_rounded'] = df['angle'].apply(round_float)

    # generate plots
    for d in [10, 15, 21, 24]:
        plot_angle(df, d, '#4169E1', '#FFA500')

    # ft
    plot_angle_ft(df_ft, '#4169E1', '#FFA500')

In [62]:
def depth(df, df_ft):
    # Round depth
    df['depth_rounded'] = df['depth'].apply(round_float)

    # generate plots
    for d in [10, 15, 21, 24]:
        plot_depth(df, d, '#3CB371', '#FFA500')

    # ft
    plot_depth_ft(df_ft, '#3CB371', '#FFA500')

In [56]:
def left_right(df, df_ft):
    # Round left/right
    df['left_right_rounded'] = df['left_right'].apply(round_float)

    # generate plots
    for d in [10, 15, 21, 24]:
        plot_left_right(df, d, '#BA55D3', '#FFA500')

    # ft
    plot_left_right_ft(df_ft, '#BA55D3', '#FFA500')

In [68]:
def violin(df, df_ft):
    # generate plots
    to_test = ['depth', 'left_right', 'angle']
    for var in to_test:
        make_violin_plot(df, var)
        make_violin_plot_ft(df_ft, var)

In [85]:
def absolute(x):
    return abs(x)

Variation

In [ ]:
def pathag(x):
    sidea, sideb = x[0], x[1]
    csquare = (sidea ** 2) + (sideb ** 2)
    sidec = math.sqrt(csquare)
    return sidec

In [ ]:
def get_player_df_by_date(player, distance_min, distance_max):
    player_df = df[(df.player_id == player)&(df.shot_distance > distance_min)&(df.shot_distance < distance_max)]
    player_df['date'] = player_df.shot_date.apply(get_date)
    player_df = player_df.sort_values('shot_date')
    player_df['floor_position_x_feet_shifted'] = player_df.floor_position_x_feet.shift(-1)
    player_df['floor_position_y_feet_shifted'] = player_df.floor_position_y_feet.shift(-1)
    player_df['x_diff'] = abs(player_df.floor_position_x_feet - player_df.floor_position_x_feet_shifted)
    player_df['y_diff'] = abs(player_df.floor_position_y_feet - player_df.floor_position_y_feet_shifted)
    player_df['movement'] = player_df[['x_diff', 'y_diff']].apply(pathag, axis=1)
    player_average = player_df[(player_df.movement < 2)&(player_df.depth > -8)&(player_df.depth < 26)&(player_df.left_right > -15)&(player_df.left_right < 15)].numeric_make_miss.mean()
    
    player_df_by_date = []
    for date in player_df.date.unique():
        date_df = player_df[player_df.date == date]
        date_df_restricted = date_df[(date_df.depth > -8)&(date_df.depth < 26)&(date_df.left_right > -15)&(date_df.left_right < 15)]
        if (len(date_df_restricted) >= 25) & (date_df_restricted.movement.mean() < 2):
            player_df_by_date.append(date_df_restricted)
    return player_df_by_date, player_average

In [ ]:
def get_player_df_by_date_with_movement(player, distance_min, distance_max):
    player_df = df[(df.player_id == player)&(df.shot_distance > distance_min)&(df.shot_distance < distance_max)]
    player_df['date'] = player_df.shot_date.apply(get_date)
    player_df = player_df.sort_values('shot_date')
    player_df['floor_position_x_feet_shifted'] = player_df.floor_position_x_feet.shift(-1)
    player_df['floor_position_y_feet_shifted'] = player_df.floor_position_y_feet.shift(-1)
    player_df['x_diff'] = abs(player_df.floor_position_x_feet - player_df.floor_position_x_feet_shifted)
    player_df['y_diff'] = abs(player_df.floor_position_y_feet - player_df.floor_position_y_feet_shifted)
    player_df['movement'] = player_df[['x_diff', 'y_diff']].apply(pathag, axis=1)
    player_average = player_df[(player_df.movement < 2)&(player_df.depth > -8)&(player_df.depth < 26)&(player_df.left_right > -15)&(player_df.left_right < 15)].numeric_make_miss.mean()
    
    player_df_by_date = []
    for date in player_df.date.unique():
        date_df = player_df[player_df.date == date]
        date_df_restricted = date_df[(date_df.depth > -8)&(date_df.depth < 26)&(date_df.left_right > -15)&(date_df.left_right < 15)]
        if (len(date_df_restricted) >= 25):
            player_df_by_date.append(date_df_restricted)
    return player_df_by_date, player_average

In [ ]:
def get_player_df_by_session(player, distance_min, distance_max, shot_count):
    player_df_by_date, player_average = get_player_df_by_date(player, distance_min, distance_max)
    player_df_by_session = []
    for date_df in player_df_by_date:
        if len(date_df) > shot_count:
            for i in range(0, len(date_df)-shot_count, shot_count):
                player_session = date_df.iloc[i:i+shot_count]
                if player_session.numeric_make_miss.mean() > 0:
                    player_df_by_session.append(player_session)
    return player_df_by_session, player_average

In [ ]:
def get_player_df_by_session_with_movement(player, distance_min, distance_max, shot_count):
    player_df_by_date, player_average = get_player_df_by_date_with_movement(player, distance_min, distance_max)
    player_df_by_session = []
    for date_df in player_df_by_date:
        if len(date_df) > shot_count:
            for i in range(0, len(date_df)-shot_count, shot_count):
                player_session = date_df.iloc[i:i+shot_count]
                if player_session.numeric_make_miss.mean() > 0:
                    player_df_by_session.append(player_session)
    return player_df_by_session, player_average

In [ ]:
def percentages_by_session(player_df_by_date, shot_count):
    percentages = []
    for date_df in player_df_by_date:
        if len(date_df) > shot_count:
            for i in range(0, len(date_df)-shot_count, shot_count):
                percentages.append(date_df.iloc[i:i+shot_count].numeric_make_miss.mean())
    percentages = pd.Series(percentages)
    return percentages

In [2]:
def get_session_df(plotting_players, shot_count):
    session_shooter_factors = []
    for player in plotting_players:
        player_df_by_session, player_average = get_player_df_by_session(player, distance_min, distance_max, shot_count)

        for session in player_df_by_session:
            stats = scores_with_correlations(session)
            session_dictionary = {'Shooting_Percentage': stats[0],
                                    'Angle_Avg': stats[1],
                                    'Angle_Cons': stats[2],
                                    'Depth_Avg': stats[3],
                                    'Depth_Cons': stats[4],
                                    'Left_Right_Avg': stats[5],
                                    'Left_Right_Cons': stats[6],
                                     'Angle_depth': stats[7],
                                     'Angle_left_Right': stats[8],
                                     'Depth_left_right':stats[9],
                                    'Player': player}
            session_dictionary['shot_count'] = len(session)
            session_shooter_factors.append(session_dictionary)
    session_df = pd.DataFrame(session_shooter_factors)
    return session_df

In [ ]:
def get_session_df_scored(plotting_players, shot_count):
    session_shooter_factors = []
    for player in plotting_players:
        player_df_by_session = get_player_df_by_session(player, distance_min, distance_max, shot_count)

        for session in player_df_by_session:
            stats = scores_by_df(session)
            session_dictionary = {'Shooting_Percentage': stats[0],
                                    'Angle_Avg': stats[1],
                                    'Angle_Cons': stats[2],
                                    'Depth_Avg': stats[3],
                                    'Depth_Cons': stats[4],
                                    'Left_Right_Avg': stats[5],
                                    'Left_Right_Cons': stats[6],
                                    'Player': player}
            session_dictionary['shot_count'] = len(session)
            session_shooter_factors.append(session_dictionary)
    session_df = pd.DataFrame(session_shooter_factors)
    return session_df

In [ ]:
def predictions_by_session(player_df_by_date, shot_count, model):
    percentages = []
    for date_df in player_df_by_date:
        if len(date_df) > shot_count:
            for i in range(0, len(date_df)-shot_count, shot_count):
                session_df = date_df.iloc[i:i+shot_count]
                # get features
                stats = scores_with_correlations(session_df)
                session_dictionary = {'sample_percentage': stats[0],
                                'Angle_Avg': stats[1],
                                'Angle_Cons': stats[2],
                                'Depth_Avg': stats[3],
                                'Depth_Cons': stats[4],
                                'Left_Right_Avg': stats[5],
                                'Left_Right_Cons': stats[6],
                                'Angle_depth': stats[7],
                                'Angle_left_Right': stats[8],
                                'Depth_left_right': stats[9],
                                'practice_percentage': player_average,
                                'player_id': player}
                # ['Angle_Avg', 'Angle_Cons', 'Depth_Avg', 'Depth_Cons', 'Left_Right_Avg', 'Left_Right_Cons', 'Angle_depth', 'Angle_left_Right', 'Depth_left_right', 'sample_percentage'
                features = list(stats[1:7]) + [stats[7][1]] + [stats[8][1]] + [stats[9][1]] + [stats[0]]
                percentages.append(model.predict(features)[0])
    percentages = pd.Series(percentages)
    return percentages

Correlations

In [ ]:
#def correlate(df, )

In [84]:
def correlations(df):
    df['left_right_abs'] = df.left_right.apply(absolute)
    
    df_restricted = df[df.shot_distance > 10]
    #print len(df_restricted)
    df_restricted = df_restricted.groupby('angle_rounded').agg(np.mean).reset_index()
    df_restricted.angle_rounded = df_restricted.angle_rounded.astype(int)
    df_restricted = df_restricted.sort('angle_rounded')
    # possibly restrict by number of shots in each category instead of by straight numbers?
    df_restricted_resonable = df_restricted[(df_restricted.angle_rounded > 35)&(df_restricted.angle_rounded < 55)]
    
    plt.scatter(df_restricted_resonable.angle_rounded, df_restricted_resonable.depth)
    plt.xlabel('Arc')
    plt.ylabel('Depth in Hoop')
    plt.show()
    #plt.savefig('./plots/other/arc_v_depth.pdf')
    '''
    plt.scatter(df_restricted_resonable.angle_rounded, df_restricted_resonable.left_right)
    plt.xlabel('Arc')
    plt.ylabel('Left/Right in Hoop')
    #plt.savefig('./plots/other/arc_v_left_right.pdf')
    plt.show()
    '''
    
    plt.scatter(df_restricted_resonable.depth, df_restricted_resonable.left_right_abs)
    plt.xlabel('Depth')
    plt.ylabel('Left/Right in Hoop')
    #plt.savefig('./plots/other/arc_v_left_right.pdf')
    plt.show()
    
    plt.scatter(df_restricted_resonable.angle_rounded, df_restricted_resonable.left_right_abs)
    plt.xlabel('Arc')
    plt.ylabel('Left/Right in Hoop')
    #plt.savefig('./plots/other/arc_v_left_right.pdf')
    plt.show()
    
    distance = 10
    color2 = '#BA55D3'
    ax = sns.countplot(x='angle_rounded', data=df[df.shot_distance > distance], order=arc_order, color=color2)
    plt.xlabel('Entry Angle')
    plt.ylabel('Number of Shots')
    plt.title('Arc vs. Number of Shots - over {0} ft'.format(str(distance)))
    #plt.savefig('../../plots/warriors/arc/number_of_shots.{0}ft.pdf'.format(str(distance)))
    plt.show()
    plt.cla()

In [82]:
def individual_correlations(df, player_id):
    p1 = (df.player_id == player_id)
    correlations(df[p1])

In [98]:
def individual_angle_distributions(df, player_id):
    p1 = (df.player_id == player_id)
    df = df[p1]
    
    #ad make miss percentage, variance and shooting percentage
    print df.make_miss.mean(), len(df[p1])
    
    d = 10
    plot_angle(df, d, '#4169E1', '#FFA500')
    plot_depth(df, d, '#3CB371', '#FFA500')
    plot_left_right(df, d, '#BA55D3', '#FFA500')


In [1]:
def individual_statistics(df, player_id):
    p1 = (df.player_id == player_id)
    df = df[p1]
    print len(df)
    
    df['left_right_abs'] = df.left_right.apply(absolute)
    
    perc_make = df.make_miss.mean()
    angle_var = df.angle.var()
    angle_mean = df.angle.mean()
    depth_var = df.depth.var()
    depth_mean = df.depth.mean()
    left_right_var = df.left_right_abs.var()
    left_right_mean = df.left_right_abs.mean()
    
    return perc_make, angle_var, angle_mean, depth_var, depth_mean, left_right_var, left_right_mean, player_id

In [4]:
def individual_statistics_abs(df, player_id):
    p1 = (df.player_id == player_id)
    df = df[p1]
    print len(df)
    
    df['left_right_abs'] = df.left_right.apply(absolute)
    df['depth_abs_distance_from_opt'] = abs(11 - df.depth)
    #df['angle_abs_distance_from_opt'] = abs(43 - df.angle)
    
    perc_make = df.make_miss.mean()
    angle_var = df.angle.var()
    #angle_abs = df.angle_abs_distance_from_opt.mean()
    depth_abs = df.depth_abs_distance_from_opt.mean()
    left_right_abs = df.left_right_abs.mean()

    return perc_make, angle_var, depth_abs, left_right_abs, player_id

In [1]:
def individual_statistics_special(df, player_id):
    p1 = (df.player_id == player_id)
    df = df[p1]
    #print len(df)
    
    df['left_right_abs_distance_median'] = abs(df.left_right.median() - df.left_right)
    df['depth_abs_distance_median'] = abs(df.depth.median() - df.depth)
    df['angle_abs_distance_median'] = abs(df.angle.median() - df.angle)
    
    perc_make = df.numeric_make_miss.mean()
    # these should be medians!!
    angle_median = df.angle.median()
    angle_cons = df.angle_abs_distance_median.median()
    depth_median = df.depth.median()
    depth_cons = df.depth_abs_distance_median.median()
    left_right_median = df.left_right.median()
    left_right_cons = df.left_right_abs_distance_median.median()

    return perc_make, angle_median, angle_cons, depth_median, depth_cons, left_right_median, left_right_cons, player_id

####Plotting

In [118]:
def plot_count(df, distance, col, color):
    number_of_shots = str(int(len(df[df.shot_distance > distance])/1000))
    ax = sns.countplot(x=col, data=df[df.shot_distance > distance], order=arc_order, color=color)
    plt.xlabel('Entry Angle')
    plt.ylabel('Number of Shots')
    plt.title('Arc vs. Number of Shots - over {0} ft'.format(str(distance)))
    plt.show()

In [3]:
def plot_angle(df, distance, color1, color2, file_out, file_out2):

    number_of_shots = str(int(len(df[df.shot_distance > distance])/1000))

    ax = sns.barplot(x='angle_rounded', y='make_miss', data=df[df.shot_distance > distance],
                     order=arc_order, color=color1, ci=None)
    plt.yticks(ax.get_yticks(), ax.get_yticks() * 100)
    plt.xlabel('Entry Angle - mean: {0}'.format(str(df[df.shot_distance > distance].angle.mean())))
    plt.ylabel('Shooting Percentage')
    plt.title('Arc vs. Shooting Percentage - {0}k shots over {1} ft'.format(number_of_shots, str(distance)))
    plt.ylim(0, 1)
    if len(file_out) > 10:
        plt.savefig(file_out)
    plt.show()
    plt.cla()

    ax = sns.countplot(x='angle_rounded', data=df[df.shot_distance > distance], order=arc_order, color=color2)
    plt.xlabel('Entry Angle')
    plt.ylabel('Number of Shots')
    plt.title('Arc vs. Number of Shots - over {0} ft'.format(str(distance)))
    if len(file_out2) > 10:
        plt.savefig(file_out2)
    plt.show()

In [44]:
def plot_angle_ft(df_ft, color1, color2):

    print 'in'
    number_of_shots = str(int(len(df_ft)/1000))
    print number_of_shots
    ax = sns.barplot(x='angle_rounded', y='make_miss', data=df_ft,
                     order=arc_order, color=color1, ci=None)
    plt.yticks(ax.get_yticks(), ax.get_yticks() * 100)
    plt.xlabel('Entry Angle - mean: {0}'.format(str(df_ft.angle.mean())))
    plt.ylabel('Shooting Percentage')
    plt.title('Arc vs. Shooting Percentage - {0}k Free Throws'.format(number_of_shots))
    plt.show()
    #plt.savefig('../../plots/mavs/arc/percentage.free_throws.pdf')
    plt.cla()

    ax = sns.countplot(x='angle_rounded', data=df_ft, order=arc_order, color=color2)
    plt.xlabel('Entry Angle')
    plt.ylabel('Number of Shots')
    plt.title('Arc vs. Number of Shots - Free Throws')
    plt.show()
    #plt.savefig('../../plots/mavs/arc/number_of_shots.free_throws.pdf')
    plt.cla()

In [18]:
def plot_depth(df, distance, color1, color2, file_out, file_out2):

    depth_order = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18']#,
               #'19', '20', '21', '22']
    #reverse(depth_order)
    
    number_of_shots = str(int(len(df[df.shot_distance > distance])/1000))

    ax = sns.barplot(x='depth_rounded', y='make_miss', data=df[df.shot_distance > distance],
                     order=depth_order, color=color1, ci=None)
    plt.yticks(ax.get_yticks(), ax.get_yticks() * 100)
    plt.xlabel('Shot Depth- mean: {0}'.format(str(df[df.shot_distance > distance].depth.mean())))
    plt.ylabel('Shooting Percentage')
    plt.title('Shot Depth vs. Shooting Percentage - {0}k shots over {1} ft'.format(number_of_shots, str(distance)))
    plt.ylim(0, 1)
    if len(file_out) > 10:
        plt.savefig(file_out)
    plt.show()
    plt.cla()

    ax = sns.countplot(x='depth_rounded', data=df[df.shot_distance > distance], order=depth_order, color=color2)
    plt.xlabel('Shot Depth')
    plt.ylabel('Number of Shots')
    plt.title('Shot Depth vs. Number of Shots - over {0} ft'.format(str(distance)))
    if len(file_out2) > 10:
        plt.savefig(file_out2)
    plt.show()


In [60]:
def plot_depth_ft(df_ft, color1, color2):

    number_of_shots = str(int(len(df_ft)/1000))

    ax = sns.barplot(x='depth_rounded', y='make_miss', data=df_ft,
                     order=depth_order, color=color1, ci=None)
    plt.yticks(ax.get_yticks(), ax.get_yticks() * 100)
    plt.xlabel('Shot Depth - mean: {0}'.format(str(df_ft.depth.mean())))
    plt.ylabel('Shooting Percentage')
    plt.title('Shot Depth vs. Shooting Percentage - {0}k Free Throws'.format(number_of_shots))
    plt.show()
    #plt.savefig('../../plots/mavs/depth/percentage.free_throws.pdf')
    plt.cla()

    ax = sns.countplot(x='depth_rounded', data=df_ft, order=depth_order, color=color2)
    plt.xlabel('Shot Depth')
    plt.ylabel('Number of Shots')
    plt.title('Shot Depth vs. Number of Shots - Free Throws')
    plt.show()
    #plt.savefig('../../plots/mavs/depth/number_of_shots.free_throws.pdf')
    plt.cla()


In [116]:
def plot_left_right(df, distance, color1, color2, file_out):

    number_of_shots = str(int(len(df[df.shot_distance > distance])/1000))

    ax = sns.barplot(x='left_right_rounded', y='make_miss', data=df[df.shot_distance > distance],
                     order=left_right_order, color=color1, ci=None)
    plt.yticks(ax.get_yticks(), ax.get_yticks() * 100)
    plt.xlabel('Left-Right - mean: {0}'.format(str(df[df.shot_distance > distance].left_right.mean())))
    plt.ylabel('Shooting Percentage')
    plt.title('Left-Right vs. Shooting Percentage - {0}k shots over {1} ft'.format(number_of_shots, str(distance)))
    plt.ylim(0, 1)
    if len(file_out) > 10:
        plt.savefig(file_out)
    plt.show()
    plt.cla()

    ax = sns.countplot(x='left_right_rounded', data=df[df.shot_distance > distance],
                       order=left_right_order, color=color2)
    plt.xlabel('Left-Right')
    plt.ylabel('Number of Shots')
    plt.title('Left-Right vs. Number of Shots - over {0} ft'.format(str(distance)))
    plt.show()



In [41]:
def plot_left_right_ft(df_ft, color1, color2):

    number_of_shots = str(int(len(df_ft)/1000))

    ax = sns.barplot(x='left_right_rounded', y='make_miss', data=df_ft,
                     order=left_right_order, color=color1, ci=None)
    plt.yticks(ax.get_yticks(), ax.get_yticks() * 100)
    plt.xlabel('Left-Right - mean: {0}'.format(str(df_ft.left_right.mean())))
    plt.ylabel('Shooting Percentage')
    plt.title('Left-Right vs. Shooting Percentage - {0}k Free Throws'.format(number_of_shots))
    plt.show()
    #plt.savefig('../../plots/mavs/left_right/percentage.free_throws.pdf')
    plt.cla()

    ax = sns.countplot(x='left_right_rounded', data=df_ft,
                       order=left_right_order, color=color2)
    plt.xlabel('Left-Right')
    plt.ylabel('Number of Shots')
    plt.title('Left-Right vs. Number of Shots - Free Throws')
    plt.show()
    #plt.savefig('../../plots/mavs/left_right/number_of_shots.free_throws.pdf')
    plt.cla()


In [42]:
def make_violin_plot(df, variable):
    sns.violinplot(data=df[(df.left_right > -15) & (df.left_right < 15)
                            & (df.depth > -5) & (df.depth < 25)
                            & (df.angle > 35) & (df.angle < 55)],
                   y=variable, x='make_miss')
    plt.xlabel('Made Shot')
    plt.ylabel(variable)
    plt.title('Distribution of {0} for Missed and Made Shots'.format(variable))
    plt.show()
    #plt.savefig('../../plots/warriors/violin/{0}.pdf'.format(variable))
    #plt.cla()

In [43]:
def make_violin_plot_ft(df_ft, variable):

        sns.violinplot(data=df_ft[(df_ft.left_right > -15) & (df_ft.left_right < 15)
                                & (df_ft.depth > -5) & (df_ft.depth < 25)
                                & (df_ft.angle > 35) & (df_ft.angle < 55)],
                       y=variable, x='make_miss')
        plt.xlabel('Made Shot')
        plt.ylabel(variable)
        plt.title('Distribution of {0} for Missed and Made Shots - Free Throws'.format(variable))
        plt.show()
        #plt.savefig('../../plots/mavs/violin/{0}.free_throws.pdf'.format(variable))
        plt.cla()


In [ ]:
# http://savvastjortjoglou.com/nba-shot-sharts.html
from matplotlib.patches import Circle, Rectangle, Arc

def draw_court(ax=None, color='black', lw=2, outer_lines=False):
    adjustx = 250
    adjusty = 50
    
    # If an axes object isn't provided to plot onto, just get current one
    if ax is None:
        ax = plt.gca()

    # Create the various parts of an NBA basketball court

    # Create the basketball hoop
    # Diameter of a hoop is 18" so it has a radius of 9", which is a value
    # 7.5 in our coordinate system
    hoop = Circle((0+adjustx, 0+adjusty), radius=7.5, linewidth=lw, color=color, fill=False)

    # Create backboard
    backboard = Rectangle((-30+adjustx, -7.5+adjusty), 60, -1, linewidth=lw, color=color)

    # The paint
    # Create the outer box 0f the paint, width=16ft, height=19ft
    outer_box = Rectangle((-80+adjustx, -47.5+adjusty), 160, 190, linewidth=lw, color=color,
                          fill=False)
    # Create the inner box of the paint, widt=12ft, height=19ft
    inner_box = Rectangle((-60+adjustx, -47.5+adjusty), 120, 190, linewidth=lw, color=color,
                          fill=False)

    # Create free throw top arc
    top_free_throw = Arc((0+adjustx, 142.5+adjusty), 120, 120, theta1=0, theta2=180,
                         linewidth=lw, color=color, fill=False)
    # Create free throw bottom arc
    bottom_free_throw = Arc((0+adjustx, 142.5+adjusty), 120, 120, theta1=180, theta2=0,
                            linewidth=lw, color=color, linestyle='dashed')
    # Restricted Zone, it is an arc with 4ft radius from center of the hoop
    restricted = Arc((0+adjustx, 0+adjusty), 80, 80, theta1=0, theta2=180, linewidth=lw,
                     color=color)

    # Three point line
    # Create the side 3pt lines, they are 14ft long before they begin to arc
    corner_three_a = Rectangle((-220+adjustx, -47.5+adjusty), 0, 140, linewidth=lw,
                               color=color)
    corner_three_b = Rectangle((220+adjustx, -47.5+adjusty), 0, 140, linewidth=lw, color=color)
    # 3pt arc - center of arc will be the hoop, arc is 23'9" away from hoop
    # I just played around with the theta values until they lined up with the 
    # threes
    three_arc = Arc((0+adjustx, 0+adjusty), 475, 475, theta1=22, theta2=158, linewidth=lw,
                    color=color)

    # Center Court
    center_outer_arc = Arc((0+adjustx, 422.5+adjusty), 120, 120, theta1=180, theta2=0,
                           linewidth=lw, color=color)
    center_inner_arc = Arc((0+adjustx, 422.5+adjusty), 40, 40, theta1=180, theta2=0,
                           linewidth=lw, color=color)

    # List of the court elements to be plotted onto the axes
    court_elements = [hoop, backboard, outer_box, inner_box, top_free_throw,
                      bottom_free_throw, restricted, corner_three_a,
                      corner_three_b, three_arc, center_outer_arc,
                      center_inner_arc]

    if outer_lines:
        # Draw the half court line, baseline and side out bound lines
        outer_lines = Rectangle((-250+adjust, -47.5+adjusty), 500, 470, linewidth=lw,
                                color=color, fill=False)
        court_elements.append(outer_lines)

    # Add the court elements onto the axes
    for element in court_elements:
        ax.add_patch(element)

    return ax